<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Закрузка-и-импорт-библиотек" data-toc-modified-id="Закрузка-и-импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Закрузка и импорт библиотек</a></span></li><li><span><a href="#Закрузка-и-проверка-данных" data-toc-modified-id="Закрузка-и-проверка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Закрузка и проверка данных</a></span></li><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Baseline" data-toc-modified-id="Baseline-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Baseline</a></span></li><li><span><a href="#Подбор-моделей" data-toc-modified-id="Подбор-моделей-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подбор моделей</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Закрузка и импорт библиотек

In [1]:
!pip install notifiers --quiet
!pip install scikit-learn --upgrade --quiet
!pip install lightgbm --quiet
!pip install progress --quiet
!pip install tqdm --quiet

In [2]:
!pip install spacy  --quiet
!python -m spacy download en_core_web_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
!pip install nltk

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import time 
from pymystem3 import Mystem
from notifiers import get_notifier
import re
import spacy 
from tqdm import tqdm, tqdm_pandas
from tqdm import notebook

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#models
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

tqdm.pandas()


In [5]:
CHAT_ID = 1234
TOKEN = '12345'

def make_notifier(token=TOKEN,
                 chat_id=CHAT_ID):
    def send(text):
        telegram=get_notifier('telegram')
        telegram.notify(message=text,
                       token=token,
                       chat_id=chat_id)
    return send

send = make_notifier()

### Закрузка и проверка данных

In [6]:
send('--->>> Машинное обучение для текстов. Начало!')

In [7]:
df_tweets = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [8]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [9]:
df_tweets.sample(10)

,text,toxic
4409,"Well, better. I will help improve your userpag...",0
18534,Deletion of Georgia's 7th congressional distri...,0
124655,Note that this page is under Pending Changes p...,0
130583,Hello there \n\nLooks like you're still retire...,0
9337,"You seem to eb paid by myspace and okcupid, th...",1
126878,YOU WILL BE ASSIMALATED,0
136313,"Also, I still think that you should have simpl...",0
20999,"Drastic change are you serious? It's outdated,...",0
30340,"More Conference reporting in April, 2016 ==\n\...",0
48218,thank you anon. I actually participated in tha...,0


In [46]:
df_tweets.pivot_table(index='toxic', values='text', aggfunc='count')

,text
toxic,
0,143346
1,16225


<b>Итог: </b> текст загружен, пропусков не обнаружено, исследован дисбаланс классов

### Очистка текста

Сначала очищаем тест от ненужных символов и сотавляем только слова

In [10]:
def clear_text(text):
    return ' '.join((re.sub(r'[^a-zA-Z]', ' ', text)).split())

In [11]:
df_tweets['text_clear'] = df_tweets['text'].progress_apply(lambda x: clear_text(x))

100%|██████████| 159571/159571 [00:04<00:00, 34734.09it/s]


### Лемматизация

Тут экспериментальным путем выяснилось, что у многих бибилиотек лемматизации довольно много недостатков - начиная с того, что они медленно работает, если лемматизировать каждый твит отдельно (выход - собирать в батчи, а потом снова разбивать после лематизации), заканчивая тем, что нормальной лемматизации для англиского языка нет (банально даже не преобразует времена глаголов).

Поэтому в проекте используется spaCy. Ее можно распараллелить штатными средствами и лемматизация получается более сносная.

In [12]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) 
send('Лемматизация...')

In [13]:
lemma = []

for doc in tqdm(nlp.pipe(df_tweets['text_clear'].astype('unicode').values, batch_size=1000, n_process=-1)):
    lemma.append(' '.join([w.lemma_ if w.lemma_ !='-PRON-' else w.text for w in doc]))

159571it [07:18, 364.10it/s] 


In [14]:
df_tweets['text_lemm'] = lemma

Проверяем результат лемматизации

In [15]:
df_tweets.iloc[0]['text_clear']

'Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalisms just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now'

In [16]:
df_tweets.iloc[0]['text_lemm']

'Explanation why the edit make under my username Hardcore Metallica Fan be revert they weren t vandalism just closure on some gas after I vote at New York Dolls FAC and please don t remove the template from the talk page since I m retire now'

<b>Итог: </b> текст очищен, лемматизация проведена. Все готово к вычислению фич и обучению моделей.

## Обучение

Рассчитаем векторы слов и сделаем из них признаки для обучения модели.

In [17]:
send('Обучение моделей...')

In [21]:
nltk.download('stopwords')
stop_words = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stop_words) 

[nltk_data] Downloading package stopwords to /home/itru/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
df_train, df_test = train_test_split(df_tweets, test_size =.2, stratify=df_tweets['toxic'])

In [23]:
corpus_train = df_train['text_lemm'].astype('U')
X_train = count_tf_idf.fit_transform(corpus_train)
y_train = df_train['toxic']

In [24]:
corpus_test = df_test['text_lemm'].astype('U')
X_test = count_tf_idf.transform(corpus_test)
y_test = df_test['toxic']

### Baseline

В качестве бейзлайна возмем самую простую и быструю Логистичекую регрессию

In [25]:
send('Baseline. Рассчет...')

In [26]:
model = LogisticRegression(n_jobs=-1)
model.fit(X_train, y_train);

In [27]:
f1_score(y_train, model.predict(X_train))

0.768530439580651

In [28]:
f1_baseline = f1_score(y_test, model.predict(X_test))

In [29]:
print('Baseline. Результат на тесте:', f1_baseline)
send('Baseline. Результат на тесте: ' + str(f1_baseline))

<b>Итог: </b> бейзлайн-модель не смогла добиться нужного порога метрики, требуются более совершенные модели.

### Подбор моделей

In [30]:
send('Обучение моделей...')

In [41]:
models = []


params_lgbm = {
    'learning_rate': [0.4, 0.5],
    'max_depth': [10, 90]#[10, 20, 60, 70, 90]
}
models.append(('LGBM', LGBMClassifier(), params_lgbm))

params_rf = {
    'n_estimators': [20,  80, 100],
    'max_depth':  [10, 30, 40]}
models.append(('Random Forest', RandomForestClassifier(), params_rf)) 


In [43]:
data_result = []

#try:
for name, model, hyperparameters in models:
    grid = GridSearchCV(model, 
                        cv=5, 
                        param_grid=hyperparameters, 
                        scoring='f1',
                        n_jobs=-1,
                        verbose=10)
    grid.fit(X_train, y_train)
    grid.best_estimator_.fit(X_train, y_train)
    predicted_test = grid.best_estimator_.predict(X_test)
    f1 = f1_score(y_test, predicted_test)
    data_result.append([f1, name, model, 
                        grid.best_params_])
    print('%s: %f' % (name, f1))
    print()
#except:
    #print('Ошибка при обучении моделей')
    #send('Ошибка при обучении моделей')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
LGBM: 0.776991

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Random Forest: 0.051621



In [53]:
result_df = pd.DataFrame(data_result, columns = ['F1','Name',
                                                   'Model','optimal_parameters']).sort_values('F1', ascending=False)

In [54]:
display(result_df)
send('Обучение моделей закончено. Лучшая модель: ' + str(result_df.iloc[0]['Model'] )+ '. RMSE:' + str(result_df.iloc[0]['F1']))

,F1,Name,Model,optimal_parameters
0,0.776991,LGBM,LGBMClassifier(),"{'learning_rate': 0.4, 'max_depth': 90}"
1,0.051621,Random Forest,RandomForestClassifier(),"{'max_depth': 40, 'n_estimators': 20}"


<b>Итог: </b> LGBM дал нужную метрику

## Выводы

В проекте мы проверили как можно классифицировать текст на основе токенов TF-IDF и достигли нужной метрики F1 0.776991 (что выше требуемого порогового значения 0,75). В процессе очистили данные от ненужного "мусора" и добились приемлимого результата лемматизации для английского текста с помощью бибилиотеки spaСy.